# Building an end-to-end Speech Recognition model in PyTorch - [AssemblyAI](https://www.assemblyai.com/)

## installing the requirements

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
!pip install torchaudio comet-ml

## Setting up your data pipeline

In [ ]:
import os
from comet_ml import Experiment
import torch
from torch.utils.data import Dataset
import torch.nn as nn
import torch.utils.data as data
import torch.optim as optim
import torch.nn.functional as F
import torchaudio
import numpy as np
import pandas as pd
import pickle
import json

In [ ]:
os.chdir('/content/drive/MyDrive/Speech_Recognition/')

In [ ]:
ANNOTATIONS_FILE = "./dataset/wavs_line_index.csv"

AUDIO_DIR = "./dataset/wavs/"


In [28]:
train_pd = pd.read_csv(ANNOTATIONS_FILE)
train_pd

,loc,label
0,nep_0258_0119737288,दीपा धामीको जन्म सुदूरपश्चिम नेपालको बझाङ जिल्...
1,nep_0258_0461984530,डिग्रा देवीको जन्म सुदूरपश्चिम नेपालको बझाङ जि...
2,nep_0258_0576399736,टेकबहादुर ऐरको जन्म सुदूरपश्चिम नेपालको डडेलधु...
3,nep_0258_0707220145,सुमन शेखरमानन्धर नेपालको कृषि अर्थविद् तथा गाय...
4,nep_0258_0838627393,आधिकारिक रूपमा पहिलो नेपाली चलचित्र नायक शिव श...
...,...,...
2059,nep_9407_9451979543,यहाँका अधिकांश जनता कृषिमै आधारित छन्
2060,nep_9407_9558473002,भीमसेनले किच्चकको बध गरे रगत पखाल्न उनले हानेक...
2061,nep_9407_9632964143,यो पत्रिकाको प्रकाशक लालचन्द्र राजभण्डारी हो
2062,nep_9407_9646005575,कालिङ्पोङमा धेरै राम्रा रेस्टुरेन्ट छन्


In [29]:
train_data = train_pd[:2000] # Pick 2000 data points
test_data = train_pd[2000:]

In [30]:
# add file path

def add_file_path(text, train = True):
  if train:
    text = AUDIO_DIR + text + ".wav"
  else:
    text = AUDIO_DIR + text + ".wav"

  return text

In [31]:
# add file path
train_data['loc'] = train_data['loc'].map(lambda x: add_file_path(x))
test_data['loc'] = test_data['loc'].map(lambda x: add_file_path(x,train= False))

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until


In [32]:
train_data

,loc,label
0,./dataset/wavs/nep_0258_0119737288.wav,दीपा धामीको जन्म सुदूरपश्चिम नेपालको बझाङ जिल्...
1,./dataset/wavs/nep_0258_0461984530.wav,डिग्रा देवीको जन्म सुदूरपश्चिम नेपालको बझाङ जि...
2,./dataset/wavs/nep_0258_0576399736.wav,टेकबहादुर ऐरको जन्म सुदूरपश्चिम नेपालको डडेलधु...
3,./dataset/wavs/nep_0258_0707220145.wav,सुमन शेखरमानन्धर नेपालको कृषि अर्थविद् तथा गाय...
4,./dataset/wavs/nep_0258_0838627393.wav,आधिकारिक रूपमा पहिलो नेपाली चलचित्र नायक शिव श...
...,...,...
1995,./dataset/wavs/nep_7957_9680379589.wav,उन्नाइस सय उनन्चासमा कोलम्बियाली फुटबल व्यावसा...
1996,./dataset/wavs/nep_7957_9826018123.wav,अष्ट्रेलियन फिल्म र टेलिभिजन अभिनेता
1997,./dataset/wavs/nep_7957_9879087796.wav,न्युफाउन्डल्यान्ड र लाब्राडोरको राजधानी सहर
1998,./dataset/wavs/nep_7957_9918407216.wav,काल्पनिक विदेशी र परिमार्जन मानव जातको सुची


In [ ]:

# with open('./models/vocab.json', 'w') as vocab_file:
#     json.dump(vocab_dict, vocab_file)
# vocab_dicts = open("./models/vocab.json", "r")
# outputs = json.loads(vocab_dicts.read())
# print((outputs))

{'क': 1, 'ख': 2, 'ग': 3, 'घ': 4, 'ङ': 5, 'च': 6, 'छ': 7, 'ज': 8, 'झ': 9, 'ञ': 10, 'ट': 11, 'ठ': 12, 'ड': 13, 'ढ': 14, 'ण': 15, 'त': 16, 'थ': 17, 'द': 18, 'ध': 19, 'न': 20, 'प': 21, 'फ': 22, 'ब': 23, 'भ': 24, 'म': 25, 'य': 26, 'र': 27, 'ल': 28, 'व': 29, 'श': 30, 'ष': 31, 'स': 32, 'ह': 33, 'क्ष': 34, 'ज्ञ': 35, 'अ': 36, 'आ': 37, 'इ': 38, 'ई': 39, 'उ': 40, 'ऊ': 41, 'ए': 42, 'ऐ': 43, 'ओ': 44, 'औ': 45, '०': 46, '१': 47, '२': 48, '३': 49, '४': 50, '५': 51, '६': 52, '७': 53, '८': 54, '९': 55, '।': 56, "'": 57, 'ं': 58, 'ँ': 59, '्': 60, 'ृ': 61, 'ू': 62, 'ी': 63, 'ौ': 64, 'ो': 65, 'ा': 66, 'ि': 67, 'ऋ': 68, '!': 69, '\u200d': 70, 'ै': 71, 'ॠ': 72, 'े': 73, '\u200c': 74, 'ः': 75, 'ु': 76, '|': 0, '[UNK]': 77, '[PAD]': 78}


In [ ]:
class MyDataset(Dataset):
 
  def __init__(self,table):
    price_df=table
 
    self.x_train=table['loc']
    self.y_train=table['label']
 
  def __len__(self):
    return len(self.y_train)
   
  def __getitem__(self,idx):
    return self.x_train[idx],self.y_train[idx]

In [23]:
def avg_wer(wer_scores, combined_ref_len):
    return float(sum(wer_scores)) / float(combined_ref_len)


def _levenshtein_distance(ref, hyp):
    """Levenshtein distance is a string metric for measuring the difference
    between two sequences. Informally, the levenshtein disctance is defined as
    the minimum number of single-character edits (substitutions, insertions or
    deletions) required to change one word into the other. We can naturally
    extend the edits to word level when calculate levenshtein disctance for
    two sentences.
    """
    m = len(ref)
    n = len(hyp)

    # special case
    if ref == hyp:
        return 0
    if m == 0:
        return n
    if n == 0:
        return m

    if m < n:
        ref, hyp = hyp, ref
        m, n = n, m

    # use O(min(m, n)) space
    distance = np.zeros((2, n + 1), dtype=np.int32)

    # initialize distance matrix
    for j in range(0,n + 1):
        distance[0][j] = j

    # calculate levenshtein distance
    for i in range(1, m + 1):
        prev_row_idx = (i - 1) % 2
        cur_row_idx = i % 2
        distance[cur_row_idx][0] = i
        for j in range(1, n + 1):
            if ref[i - 1] == hyp[j - 1]:
                distance[cur_row_idx][j] = distance[prev_row_idx][j - 1]
            else:
                s_num = distance[prev_row_idx][j - 1] + 1
                i_num = distance[cur_row_idx][j - 1] + 1
                d_num = distance[prev_row_idx][j] + 1
                distance[cur_row_idx][j] = min(s_num, i_num, d_num)

    return distance[m % 2][n]


def word_errors(reference, hypothesis, ignore_case=False, delimiter=' '):
    """Compute the levenshtein distance between reference sequence and
    hypothesis sequence in word-level.
    :param reference: The reference sentence.
    :type reference: basestring
    :param hypothesis: The hypothesis sentence.
    :type hypothesis: basestring
    :param ignore_case: Whether case-sensitive or not.
    :type ignore_case: bool
    :param delimiter: Delimiter of input sentences.
    :type delimiter: char
    :return: Levenshtein distance and word number of reference sentence.
    :rtype: list
    """
    if ignore_case == True:
        reference = reference.lower()
        hypothesis = hypothesis.lower()

    ref_words = reference.split(delimiter)
    hyp_words = hypothesis.split(delimiter)

    edit_distance = _levenshtein_distance(ref_words, hyp_words)
    return float(edit_distance), len(ref_words)


def char_errors(reference, hypothesis, ignore_case=False, remove_space=False):
    """Compute the levenshtein distance between reference sequence and
    hypothesis sequence in char-level.
    :param reference: The reference sentence.
    :type reference: basestring
    :param hypothesis: The hypothesis sentence.
    :type hypothesis: basestring
    :param ignore_case: Whether case-sensitive or not.
    :type ignore_case: bool
    :param remove_space: Whether remove internal space characters
    :type remove_space: bool
    :return: Levenshtein distance and length of reference sentence.
    :rtype: list
    """
    if ignore_case == True:
        reference = reference.lower()
        hypothesis = hypothesis.lower()

    join_char = ' '
    if remove_space == True:
        join_char = ''

    reference = join_char.join(filter(None, reference.split(' ')))
    hypothesis = join_char.join(filter(None, hypothesis.split(' ')))

    edit_distance = _levenshtein_distance(reference, hypothesis)
    return float(edit_distance), len(reference)


def wer(reference, hypothesis, ignore_case=False, delimiter=' '):
    """Calculate word error rate (WER). WER compares reference text and
    hypothesis text in word-level. WER is defined as:
    .. math::
        WER = (Sw + Dw + Iw) / Nw
    where
    .. code-block:: text
        Sw is the number of words subsituted,
        Dw is the number of words deleted,
        Iw is the number of words inserted,
        Nw is the number of words in the reference
    We can use levenshtein distance to calculate WER. Please draw an attention
    that empty items will be removed when splitting sentences by delimiter.
    :param reference: The reference sentence.
    :type reference: basestring
    :param hypothesis: The hypothesis sentence.
    :type hypothesis: basestring
    :param ignore_case: Whether case-sensitive or not.
    :type ignore_case: bool
    :param delimiter: Delimiter of input sentences.
    :type delimiter: char
    :return: Word error rate.
    :rtype: float
    :raises ValueError: If word number of reference is zero.
    """
    edit_distance, ref_len = word_errors(reference, hypothesis, ignore_case,
                                         delimiter)

    if ref_len == 0:
        raise ValueError("Reference's word number should be greater than 0.")

    wer = float(edit_distance) / ref_len
    return wer


def cer(reference, hypothesis, ignore_case=False, remove_space=False):
    """Calculate charactor error rate (CER). CER compares reference text and
    hypothesis text in char-level. CER is defined as:
    .. math::
        CER = (Sc + Dc + Ic) / Nc
    where
    .. code-block:: text
        Sc is the number of characters substituted,
        Dc is the number of characters deleted,
        Ic is the number of characters inserted
        Nc is the number of characters in the reference
    We can use levenshtein distance to calculate CER. Chinese input should be
    encoded to unicode. Please draw an attention that the leading and tailing
    space characters will be truncated and multiple consecutive space
    characters in a sentence will be replaced by one space character.
    :param reference: The reference sentence.
    :type reference: basestring
    :param hypothesis: The hypothesis sentence.
    :type hypothesis: basestring
    :param ignore_case: Whether case-sensitive or not.
    :type ignore_case: bool
    :param remove_space: Whether remove internal space characters
    :type remove_space: bool
    :return: Character error rate.
    :rtype: float
    :raises ValueError: If the reference length is zero.
    """
    edit_distance, ref_len = char_errors(reference, hypothesis, ignore_case,
                                         remove_space)

    if ref_len == 0:
        raise ValueError("Length of reference should be greater than 0.")

    cer = float(edit_distance) / ref_len
    return cer

class TextTransform:
    """Maps characters to integers and vice versa"""
    def __init__(self):
        # char_map_str = """
        #   <SPACE> 0
        #   क 1
        #   ख 2
        #   ग 3
        #   घ 4
        #   ङ 5
        #   च 6
        #   छ 7
        #   ज 8
        #   झ 9
        #   ञ 10
        #   ट 11
        #   ठ 12
        #   ड 13
        #   ढ 14
        #   ण 15
        #   त 16
        #   थ 17
        #   द 18
        #   ध 19
        #   न 20
        #   प 21
        #   फ 22
        #   ब 23
        #   भ 24
        #   म 25
        #   य 26
        #   र 27
        #   ल 28
        #   व 29
        #   श 30
        #   ष 31
        #   स 32
        #   ह 33
        #   अ 34
        #   आ 35
        #   इ 36
        #   ई 37
        #   उ 38
        #   ऊ 39
        #   ए 40
        #   ऐ 41
        #   ओ 42
        #   औ 43
        #    ा 44
        #    ि 45
        #    ी 46
        #    ु 47
        #    ू 48
        #    ृ 49
        #    े 50
        #    ै 51
        #    ो 52
        #    ौ 53
        #    ् 54
        #    ँ 55
        #    ं 56
        #   ० 57
        #   १ 58
        #   २ 59
        #   ३ 60
        #   ४ 61
        #   ५ 62
        #   ६ 63
        #   ७ 64
        #   ८ 65
        #   ९ 66
        #   । 67
        #   ' 68
        #   """
        vocab_dict = open("./models/vocab.json", "r")
        self.output = json.loads(vocab_dict.read())
      
        
        self.char_map = {}
        self.index_map = {}
        self.char_map[' '] = 79
        self.index_map[79] = ' '
        for key in self.output:
            self.char_map[key] = int(self.output[key])
            self.index_map[int(self.output[key])] = key
        

    def text_to_int(self, text):
        """ Use a character map and convert text to an integer sequence """
        int_sequence = []
        for c in text:
            # if c == ' ':
            #     ch = self.char_map['<SPACE>']
            # else:

            if c in list(self.output):
              ch = self.char_map[c]
              int_sequence.append(ch)
        return int_sequence

    def int_to_text(self, labels):
        """ Use a character map and convert integer labels to an text sequence """
        string = []
        for i in labels:
            string.append(self.index_map[i])
        return ''.join(string).replace('', ' ')

        # .replace('<SPACE>', ' ')

train_audio_transforms = nn.Sequential(
    torchaudio.transforms.Resample(48000,16000),
    torchaudio.transforms.MelSpectrogram(sample_rate=16000, n_mels=128),
    torchaudio.transforms.FrequencyMasking(freq_mask_param=15),
    torchaudio.transforms.TimeMasking(time_mask_param=35)
)

valid_audio_transforms =train_audio_transforms = nn.Sequential(
    torchaudio.transforms.Resample(48000,16000),
    torchaudio.transforms.MelSpectrogram(sample_rate=16000),
    torchaudio.transforms.FrequencyMasking(freq_mask_param=15),
    torchaudio.transforms.TimeMasking(time_mask_param=35)
)

text_transform = TextTransform()

def data_processing(data,data_type="train"):
    spectrograms = []
    labels = []
    input_lengths = []
    label_lengths = []
    for (wave, label) in data:
        wav, sampling_rate = torchaudio.load(wave)
        
        if data_type == 'train':
            spec = train_audio_transforms(wav).squeeze(0).transpose(0, 1)
        else:
            spec = valid_audio_transforms(wav).squeeze(0).transpose(0, 1)
        spectrograms.append(spec)
        
        
        label = torch.Tensor(text_transform.text_to_int(label))
        

        labels.append(label)
        input_lengths.append(spec.shape[0]//2)
        label_lengths.append(len(label))

    spectrograms = nn.utils.rnn.pad_sequence(spectrograms, batch_first=True).unsqueeze(1).transpose(2, 3)
    labels = nn.utils.rnn.pad_sequence(labels, batch_first=True)

    return spectrograms, labels, input_lengths, label_lengths


def GreedyDecoder(output, labels, label_lengths, blank_label=0, collapse_repeated=True):
	arg_maxes = torch.argmax(output, dim=2)
	decodes = []
	targets = []
	for i, args in enumerate(arg_maxes):
		decode = []
		targets.append(text_transform.int_to_text(labels[i][:label_lengths[i]].tolist()))
		for j, index in enumerate(args):
			if index != blank_label:
				if collapse_repeated and j != 0 and index == args[j -1]:
					continue
				decode.append(index.item())
		decodes.append(text_transform.int_to_text(decode))
	return decodes, targets


/usr/local/lib/python3.7/dist-packages/torchaudio/functional/functional.py:595: UserWarning: At least one mel filterbank has all zero values. The value for `n_mels` (128) may be set too high. Or, the value for `n_freqs` (201) may be set too low.
  "At least one mel filterbank has all zero values. "


## The Model
Base of of Deep Speech 2 with some personal improvements

In [ ]:
class CNNLayerNorm(nn.Module):
    """Layer normalization built for cnns input"""
    def __init__(self, n_feats):
        super(CNNLayerNorm, self).__init__()
        self.layer_norm = nn.LayerNorm(n_feats)

    def forward(self, x):
        # x (batch, channel, feature, time)
        x = x.transpose(2, 3).contiguous() # (batch, channel, time, feature)
        x = self.layer_norm(x)
        return x.transpose(2, 3).contiguous() # (batch, channel, feature, time) 


class ResidualCNN(nn.Module):
    """Residual CNN inspired by https://arxiv.org/pdf/1603.05027.pdf
        except with layer norm instead of batch norm
    """
    def __init__(self, in_channels, out_channels, kernel, stride, dropout, n_feats):
        super(ResidualCNN, self).__init__()

        self.cnn1 = nn.Conv2d(in_channels, out_channels, kernel, stride, padding=kernel//2)
        self.cnn2 = nn.Conv2d(out_channels, out_channels, kernel, stride, padding=kernel//2)
        self.dropout1 = nn.Dropout(dropout)
        self.dropout2 = nn.Dropout(dropout)
        self.layer_norm1 = CNNLayerNorm(n_feats)
        self.layer_norm2 = CNNLayerNorm(n_feats)

    def forward(self, x):
        residual = x  # (batch, channel, feature, time)
        x = self.layer_norm1(x)
        x = F.gelu(x)
        x = self.dropout1(x)
        x = self.cnn1(x)
        x = self.layer_norm2(x)
        x = F.gelu(x)
        x = self.dropout2(x)
        x = self.cnn2(x)
        x += residual
        return x # (batch, channel, feature, time)


class BidirectionalGRU(nn.Module):

    def __init__(self, rnn_dim, hidden_size, dropout, batch_first):
        super(BidirectionalGRU, self).__init__()

        self.BiGRU = nn.GRU(
            input_size=rnn_dim, hidden_size=hidden_size,
            num_layers=1, batch_first=batch_first, bidirectional=True)
        self.layer_norm = nn.LayerNorm(rnn_dim)
        self.dropout = nn.Dropout(dropout)

    def forward(self, x):
        x = self.layer_norm(x)
        x = F.gelu(x)
        x, _ = self.BiGRU(x)
        x = self.dropout(x)
        return x


class SpeechRecognitionModel(nn.Module):
    
    def __init__(self, n_cnn_layers, n_rnn_layers, rnn_dim, n_class, n_feats, stride=2, dropout=0.1):
        super(SpeechRecognitionModel, self).__init__()
        n_feats = n_feats//2
        self.cnn = nn.Conv2d(1, 32, 3, stride=stride, padding=3//2)  # cnn for extracting heirachal features

        # n residual cnn layers with filter size of 32
        self.rescnn_layers = nn.Sequential(*[
            ResidualCNN(32, 32, kernel=3, stride=1, dropout=dropout, n_feats=n_feats) 
            for _ in range(n_cnn_layers)
        ])
        self.fully_connected = nn.Linear(n_feats*32, rnn_dim)
        self.birnn_layers = nn.Sequential(*[
            BidirectionalGRU(rnn_dim=rnn_dim if i==0 else rnn_dim*2,
                             hidden_size=rnn_dim, dropout=dropout, batch_first=i==0)
            for i in range(n_rnn_layers)
        ])
        self.classifier = nn.Sequential(
            nn.Linear(rnn_dim*2, rnn_dim),  # birnn returns rnn_dim*2
            nn.GELU(),
            nn.Dropout(dropout),
            nn.Linear(rnn_dim, n_class)
        )

    def forward(self, x):
        x = self.cnn(x)
        x = self.rescnn_layers(x)
        sizes = x.size()
        x = x.view(sizes[0], sizes[1] * sizes[2], sizes[3])  # (batch, feature, time)
        x = x.transpose(1, 2) # (batch, time, feature)
        x = self.fully_connected(x)
        x = self.birnn_layers(x)
        x = self.classifier(x)
        return x


## The Training and Evaluating Script

In [24]:
class IterMeter(object):
    """keeps track of total iterations"""
    def __init__(self):
        self.val = 0

    def step(self):
        self.val += 1

    def get(self):
        return self.val


def train(model, device, train_loader, criterion, optimizer, scheduler, epoch, iter_meter, experiment):
    model.train()
    data_len = len(train_loader.dataset)
    with experiment.train():
        for batch_idx, _data in enumerate(train_loader):
            spectrograms, labels, input_lengths, label_lengths = _data 
            spectrograms=spectrograms.to(device)
            # labels = np.asarray(labels)
            # labels = torch.from_numpy(labels)
            labels =  labels.to(device)

            optimizer.zero_grad()

            output = model(spectrograms)  # (batch, time, n_class)
            output = F.log_softmax(output, dim=2)
            output = output.transpose(0, 1) # (time, batch, n_class)

            loss = criterion(output, labels, input_lengths, label_lengths)
            loss.backward()

            
            experiment.log_metric('loss', loss.item(), step=iter_meter.get())
            experiment.log_metric('learning_rate', scheduler.get_lr(), step=iter_meter.get())

            optimizer.step()
            scheduler.step()
            iter_meter.step()
            if batch_idx % 100 == 0 or batch_idx == data_len:
                print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                    epoch, batch_idx * len(spectrograms), data_len,
                    100. * batch_idx / len(train_loader), loss.item()))
                
    return model


def test(model, device, test_loader, criterion, epoch, iter_meter, experiment):
    print('\nevaluating...')
    model.eval()
    test_loss = 0
    test_cer, test_wer = [], []
    with experiment.test():
        with torch.no_grad():
            for i, _data in enumerate(test_loader):
                spectrograms, labels, input_lengths, label_lengths = _data 
                spectrograms, labels = spectrograms.to(device), labels.to(device)

                output = model(spectrograms)  # (batch, time, n_class)
                output = F.log_softmax(output, dim=2)
                output = output.transpose(0, 1) # (time, batch, n_class)

                loss = criterion(output, labels, input_lengths, label_lengths)
                test_loss += loss.item() / len(test_loader)

                decoded_preds, decoded_targets = GreedyDecoder(output.transpose(0, 1), labels, label_lengths)
                for j in range(len(decoded_preds)):
                    test_cer.append(cer(decoded_targets[j], decoded_preds[j]))
                    test_wer.append(wer(decoded_targets[j], decoded_preds[j]))


    avg_cer = sum(test_cer)/len(test_cer)
    avg_wer = sum(test_wer)/len(test_wer)
    experiment.log_metric('test_loss', test_loss, step=iter_meter.get())
    experiment.log_metric('cer', avg_cer, step=iter_meter.get())
    experiment.log_metric('wer', avg_wer, step=iter_meter.get())

    print('Test set: Average loss: {:.4f}, Average CER: {:4f} Average WER: {:.4f}\n'.format(test_loss, avg_cer, avg_wer))


def main(learning_rate=5e-4, batch_size=20, epochs=10,experiment=Experiment(api_key='dummy_key', disabled=True)):

    hparams = {
        "n_cnn_layers": 3,
        "n_rnn_layers": 5,
        "rnn_dim": 512,
        "n_class": 80,
        "n_feats": 128,
        "stride":2,
        "dropout": 0.1,
        "learning_rate": learning_rate,
        "batch_size": batch_size,
        "epochs": epochs
    }

    experiment.log_parameters(hparams)

    use_cuda = torch.cuda.is_available()
    torch.manual_seed(7)
    device = torch.device("cuda" if use_cuda else "cpu")

    if not os.path.isdir("./data"):
        os.makedirs("./data")

    train_dataset = MyDataset(train_data)
    test_dataset = MyDataset(test_data)

    kwargs = {'num_workers': 1, 'pin_memory': True} if use_cuda else {}
    train_loader = data.DataLoader(dataset=train_dataset,
                                batch_size=hparams['batch_size'],
                                shuffle=True,
                                collate_fn=lambda x: data_processing(x, 'train'),
                                **kwargs)
    
    test_loader = data.DataLoader(dataset=test_dataset,
                                batch_size=hparams['batch_size'],
                                shuffle=True,
                                collate_fn=lambda x: data_processing(x, 'test'),
                                **kwargs)
    

    model = SpeechRecognitionModel(
        hparams['n_cnn_layers'], hparams['n_rnn_layers'], hparams['rnn_dim'],
        hparams['n_class'], hparams['n_feats'], hparams['stride'], hparams['dropout']
        ).to(device)

    print(model)
    print('Num Model Parameters', sum([param.nelement() for param in model.parameters()]))

    optimizer = optim.AdamW(model.parameters(), hparams['learning_rate'])
    criterion = nn.CTCLoss(blank=0, zero_infinity=False).to(device)
    scheduler = optim.lr_scheduler.OneCycleLR(optimizer, max_lr=hparams['learning_rate'], 
                                            steps_per_epoch=int(len(train_loader)),
                                            epochs=hparams['epochs'],
                                            anneal_strategy='linear')
    
    iter_meter = IterMeter()

    
    for epoch in range(1, epochs + 1):
        model =train(model, device, train_loader, criterion, optimizer, scheduler, epoch, iter_meter, experiment)
    pickle.dump(model, open('./nep_speech_model.pkl', 'wb'))
    test(model, device, train_loader, criterion, 1, iter_meter, experiment)    


COMET WARNING: As you are running in a Jupyter environment, you will need to call `experiment.end()` when finished to ensure all metrics and code are logged before exiting.


## Setting up Comet
If you have a comet account, fill in teh api key, project name and experiment name below. You can create an account at [comet.ml](comet.ml).

In [ ]:
comet_api_key = "VSrB9HwR33pCfDpJIrNBB7coh"
workspace="shawn00-tech"
project_name = "speechrecognition"
experiment_name = "speechrecognition-colab"

if comet_api_key:
  experiment = Experiment(api_key=comet_api_key, project_name=project_name, parse_args=False)
  experiment.set_name(experiment_name)
  experiment.display()
else:
  experiment = Experiment(api_key='dummy_key', disabled=True)

COMET WARNING: As you are running in a Jupyter environment, you will need to call `experiment.end()` when finished to ensure all metrics and code are logged before exiting.
COMET INFO: Couldn't find a Git repository in '/content/drive/MyDrive/Colab_Notebooks/sound/Speech_Recognition' nor in any parent directory. You can override where Comet is looking for a Git Patch by setting the configuration `COMET_GIT_DIRECTORY`
COMET INFO: Experiment is live on comet.ml https://www.comet.ml/shawn00-tech/speechrecognition/654e23598a514126ac184214072eede0



## GPU runtime
If you are using a GPU runtime, this will let you know what GPU and how much memory is available. Adjust your batch_size depending on which GPU

In [ ]:
!nvidia-smi

NVIDIA-SMI has failed because it couldn't communicate with the NVIDIA driver. Make sure that the latest NVIDIA driver is installed and running.



## Train
this will download the data on first run and may take a while. 

If you have Comet.ml setup, you can start seeing your progress in the comet cell above.

In [ ]:
learning_rate = 5e-4
batch_size = 10
epochs = 10


main(learning_rate, batch_size, epochs,experiment)

SpeechRecognitionModel(
  (cnn): Conv2d(1, 32, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1))
  (rescnn_layers): Sequential(
    (0): ResidualCNN(
      (cnn1): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (cnn2): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (dropout1): Dropout(p=0.1, inplace=False)
      (dropout2): Dropout(p=0.1, inplace=False)
      (layer_norm1): CNNLayerNorm(
        (layer_norm): LayerNorm((64,), eps=1e-05, elementwise_affine=True)
      )
      (layer_norm2): CNNLayerNorm(
        (layer_norm): LayerNorm((64,), eps=1e-05, elementwise_affine=True)
      )
    )
    (1): ResidualCNN(
      (cnn1): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (cnn2): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (dropout1): Dropout(p=0.1, inplace=False)
      (dropout2): Dropout(p=0.1, inplace=False)
      (layer_norm1): CNNLayerNorm(
        (layer_norm): LayerNorm((64,),

/usr/local/lib/python3.7/dist-packages/torch/optim/lr_scheduler.py:1565: UserWarning: To get the last learning rate computed by the scheduler, please use `get_last_lr()`.
  "please use `get_last_lr()`.", UserWarning)


Train Epoch: 1 [0/2000 (0%)]	Loss: 14.903978


In [ ]:
with open('./nep_speech_model.pkl', 'rb') as f:
    model = pickle.load(f)

In [ ]:
train_dataset = MyDataset(train_data)

train_loader = data.DataLoader(dataset=train_dataset,
                                batch_size=5,
                                shuffle=True,
                                collate_fn=lambda x: data_processing(x, 'test'))

In [ ]:
model.eval()
test_loss = 0
test_cer, test_wer = [], []
with experiment.test():
  with torch.no_grad():
    for i, _data in enumerate(train_loader):
      spectrograms, labels, input_lengths, label_lengths = _data 
      spectrograms, labels = spectrograms.to(device), labels.to(device)

      output = model(spectrograms)  # (batch, time, n_class)
      output = F.log_softmax(output, dim=2)
      output = output.transpose(0, 1) # (time, batch, n_class)

      loss = criterion(output, labels, input_lengths, label_lengths)
      test_loss += loss.item() / len(test_loader)

      decoded_preds, decoded_targets = GreedyDecoder(output.transpose(0, 1), labels, label_lengths)
      for j in range(len(decoded_preds)):
        test_cer.append(cer(decoded_targets[j], decoded_preds[j]))
        test_wer.append(wer(decoded_targets[j], decoded_preds[j]))


avg_cer = sum(test_cer)/len(test_cer)
avg_wer = sum(test_wer)/len(test_wer)
print('Test set: Average loss: {:.4f}, Average CER: {:4f} Average WER: {:.4f}\n'.format(test_loss, avg_cer, avg_wer))


NameError: ignored